# Non-Ergodic GMM Regression (Type-2) using STAN, PyStan Library
## Partially Spatially Correlated Anelastic Attenuation Cells, <br>  using sparse cell-path distance matrix

This notebook is used to estimate the non-ergodic coefficients, anelastic attenuation coefficients, and hyper-parameters of a type-2 non-ergodic GMM though Gaussian Process regression using the Bayesian software STAN. 

The Type-2 GMM compnents, with correlated anelastic attenuation cells, are:
- The non-ergodic effects are modeled by:
  - $\delta c_{0}$:    constant shift of non-ergodic GMM with respect to base ergodic GMM
  - $\delta c_{1,e}$:  spatially varying eathquake adjustment as a function of the earthquake coordinates
  - $\delta c_{1a,s}$: spatially varying site adjustment as a function of the site coordinates
  - $\delta c_{1b,s}$: spatially independent site adjustment as a function of the station id
  - $c_{ca,p}$:        cell-specific anelastic attenuation
- The aleatory variability is captured by:
  - $\delta B^0_{e}$: between-event aleatory term
  - $\delta W^0_{e,s}$: within-event aleatory term
- The non-ergodic hyperparameters are:
  - $\ell_{1,e}$:     correlation lenght of spatially varying earthquake adjustment
  - $\ell_{1a,s}$:    correlation lenght of spatially varying site adjustment
  - $\omega_{1,e}$:   scale of spatially varying earthquake adjustment 
  - $\omega_{1a,s}$:  scale of spatially varying site adjustment
  - $\omega_{1b,s}$:  scale of spatially independent site adjustment
  - $\ell_{ca1,p}$:   mean of anelastic attenuation cells
  - $\ell_{ca1,p}$:   correlation length of spatially varying component of anelastic attenuation cells
  - $\omega_{ca1,p}$: scale of spatially varying component of anelastic attenuation cell coefficients
  - $\omega_{ca2,p}$: scale of spatially independent component of anelastic attenuation cell coefficients
  - $\tau_{0}$:       standard deviation of between-event aleatory
  - $\phi_{0}$:       standard deviation of within-event aleatory

**Disclaimer:** The non-ergodic regression is perfomed on the total residuals (column ``res_name``) of a base ergodic GMM without including the effect of anelastic attenuation. 
The total ergodic residuals ($e'_{tot}$) without the effect of anelastic attenuation are defined as:
$$
e'_{tot} = log(IM) - ((\mu_{erg}(M,R_{rup},V_{S30}, ...) - c_{a~erg} ~ R_{rup})
$$
where $IM$ is the intesity parameter of interest (e.g. PGA, PSA), $\mu_{erg}$ is mean functional form of the ergodic GMM in log space, and $c_{a~erg}$ is the ergodic anelastic attenuation coefficient.

## Load required libraries

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import time
#user functions
sys.path.insert(0,'../Python_lib/regression/pystan')
from regression_pystan_model2_corr_cells_sparse_unbounded_hyp import RunStan

## User Input, Definition of Regression Parameters 
The 1st synthetic dataset based on the North CA subset of NGAWest2 is used as an example; however, the user should update the variables in this section to correspond to the dataset of interest. 
The entire 1st NGAWest3 CA syntethic dataset is loaded in this section. In the preprocessing section only the records that belong in the the Northern part of CA in NGAWest2 are kept.

In [5]:
#regression name
reg_name  = 'example_regression_ds2'

#regression dataset
flatfile_fname = '../../Data/Validation/synthetic_datasets/ds2_small_corr_len/CatalogNGAWest3CALite_synthetic_data_small_corr_len_Y1.csv'
#cell-path info and distance matrix
cellinfo_fname    = '../../Data/Validation/synthetic_datasets/ds2_small_corr_len/CatalogNGAWest3CALite_cellinfo.csv'
celldist_sp_fname = '../../Data/Validation/synthetic_datasets/ds2_small_corr_len/CatalogNGAWest3CALite_distancematrix_sparse.csv'

#output directory
dir_out = '../../Data/Regression/example_ds2_sp/'

#filename for stan regression code
m_fname = '../Python_lib/regression/stan/regression_stan_model2_corr_cells_sparse_unbounded_hyp_chol_efficient.stan'

#STAN parameters
runstan_flag = True
pystan_ver = 2
res_name = 'tot'
n_iter = 1000
n_iter = 10 #################
n_chains = 4
adapt_delta   = 0.8 #0.9
max_treedepth = 10
#parallel options
# flag_parallel = True
flag_parallel = False
#ergodic anelastic attenutation
c_a_erg = 0

## Load Files

In [10]:
#load flatfile
df_flatfile = pd.read_csv(flatfile_fname)

#load cell dataframes
df_cellinfo    = pd.read_csv(cellinfo_fname)
df_celldist_sp = pd.read_csv(celldist_sp_fname)

In [11]:
df_celldist_sp

,row,col,data
0,1,785,0.796186
1,1,786,26.485917
2,1,787,10.188899
3,2,786,22.808630
4,2,787,11.219578
...,...,...,...
1628700,157438,1445,27.306674
1628701,157438,1493,9.553922
1628702,157438,1494,29.356114
1628703,157438,1495,8.876376


In [20]:
from scipy import sparse

n_data = len(df_flatfile)
n_cell = len(df_cellinfo)

sparse.coo_matrix((df_celldist_sp.data.values, (df_celldist_sp.row.values-1, df_celldist_sp.col.values-1)),
                 shape=(n_data, n_cell))



<157438x2040 sparse matrix of type '<class 'numpy.float64'>'
	with 1628705 stored elements in COOrdinate format>

## Preprocessing 
This section can be used by the user for any pre-porcessing of their dataset.

In [4]:
#keep only North records of NGAWest2
df_flatfile = df_flatfile.loc[np.logical_and(df_flatfile.dsid==0,
                                             df_flatfile.sreg==1),:]


## Run Regression

In [7]:
print(f'Running regression for:%s'%(reg_name))

#run time start
run_t_strt = time.time()

#run stan model
RunStan(df_flatfile, df_cellinfo, df_celldist, sm_fname, 
        reg_name, dir_out, res_name, c_a_erg=c_a_erg, 
        runstan_flag=runstan_flag, n_iter=n_iter, n_chains=n_chains,
        adapt_delta=adapt_delta, max_treedepth=max_treedepth,
        pystan_ver=pystan_ver, pystan_parallel=flag_parallel)
       
#run time end
run_t_end = time.time()

#compute run time
run_tm = (run_t_end - run_t_strt)/60
  
#log run time
df_run_info = pd.DataFrame({'computer_name':os.uname()[1],'reg_name':reg_name,'run_time':run_tm}, 
                            index=[1])
                           
#write out run info
fname_reginfo   = '%s/run_info.csv'%(dir_out)
df_run_info.reset_index(drop=True).to_csv(fname_reginfo, index=False)

print(f'Completed regression for: %s'%(reg_name))

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_6113a8804c0077e2dc53c6f5b7a64f58 NOW.


Running regression for:example_regression_ds2
max R_rup misfit 1.3073986337985843e-12


cc1plus: warning: command line option '-Wstrict-prototypes' is valid for C/ObjC but not for C++
In file included from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /tmp/pystan_kf_jj3n7/stanfit4anon_model_6113a8804c0077e2dc53c6f5b7a64f58_3531452570514397354.cpp:771:
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
In file included from /ho

In file included from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/src/stan/io/dump.hpp:6,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/py_var_context.hpp:12,
                 from /tmp/pystan_kf_jj3n7/stanfit4anon_model_6113a8804c0077e2dc53c6f5b7a64f58_3531452570514397354.cpp:78

In file included from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Dense:1,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat/meta/as_array_or_scalar.hpp:4,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/lib/stan_math/stan/math/prim/mat.hpp:9,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/stan/src/stan/io/dump.hpp:6,
                 from /home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/pystan/py_var_context.hpp:12,
                 from /tmp/pystan_kf_jj3n7/stanfit4anon_model_6113a8804c0077e2dc53c6f5b7a64f58_3531452570514397354.cpp:78

...


 Elapsed Time: 2.77283 seconds (Warm-up)
               1.26193 seconds (Sampling)
               4.03475 seconds (Total)



/mnt/halcloud_nfs/glavrent/Research/Nonerg_GMM_methodology/Analyses/Regression/../Python_lib/regression/pystan/regression_pystan_model2_corr_cells_unbounded_hyp.py:198: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump({'model' : stan_model, 'fit' : stan_fit}, f, protocol=-1)
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: divide by zero encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: invalid value encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: divide by zero encountered in true_divide
  pdf 

/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: divide by zero encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: invalid value encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: divide by zero encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: invalid value encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: divide by zero encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavre

/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:620: RuntimeWarning: invalid value encountered in double_scalars
  x_std = (((x ** 2).sum() / x_len) - (x.sum() / x_len) ** 2) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:770: UserWarning: Something failed when estimating the bandwidth. Please check your data
  warnings.warn("Something failed when estimating the bandwidth. Please check your data")
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: divide by zero encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/density_utils.py:783: RuntimeWarning: invalid value encountered in true_divide
  pdf /= bw * (2 * np.pi) ** 0.5
/home/glavrent/anaconda3/envs/python_3.8stan/lib/python3.8/site-packages/arviz/stats/den

Completed regression for: example_regression_ds2
